## Testing the electrophysiological properties : NFS neurons

In [1]:
%matplotlib notebook

import brian2 as b2
import numpy
import random
import matplotlib.pyplot as plt
import pandas as pd
import datetime

from input_factory import get_step_current, get_ou_current
from plot_tools import plot_voltage_and_current_traces

In [2]:
model_eqs = '''
dv/dt = (-gL*(v-v_rest) - w1 - w2 + I_ext(t))/C : volt

dw1/dt = -w1/tau_w1 : amp
dw2/dt = -w2/tau_w2 : amp

dvt1/dt = -vt1/tau_vt1 : volt
dvt2/dt = -vt2/tau_vt2 : volt
 
vt = v_thresh + vt1 + vt2 : volt
lambda_t = lambda_0*exp((v-vt)/del_v): Hz
'''
reset_eqs = '''
v = v_reset
w1+=amp_w1
w2+=amp_w2
vt1+=amp_vt1
vt2+=amp_vt2
'''

In [3]:
nfs_df = pd.read_csv('data/nfs.txt',header = None)
nfs_df.columns =['C (nF)','gL (usiemens)','v_rest (mV)','v_reset (mV)','tau_refr (ms)',\
             'v_thresh (mV)','del_v (mV)', 'amp_w1 (nA)','tau_w1 (ms)','amp_w2 (nA)',\
             'tau_w2 (ms)','amp_vt1 (mV)','tau_vt1 (ms)','amp_vt2 (mV)','tau_vt2 (ms)']
fs_df = pd.read_csv('data/fs.txt',header = None)
fs_df.columns =['C (nF)','gL (usiemens)','v_rest (mV)','v_reset (mV)','tau_refr (ms)',\
             'v_thresh (mV)','del_v (mV)', 'amp_w1 (nA)','tau_w1 (ms)','amp_w2 (nA)',\
             'tau_w2 (ms)','amp_vt1 (mV)','tau_vt1 (ms)','amp_vt2 (mV)','tau_vt2 (ms)']

In [4]:
nfs_df

In [5]:
print('R_in_nfs',1/nfs_df.iloc[0,:][1])
print('tau_nfs',nfs_df.iloc[0,:][0]/nfs_df.iloc[0,:][1])

In [6]:
b2.start_scope()
nfs_param = nfs_df.iloc[0,:] #set which value to try out, set any if running through all
C = nfs_param['C (nF)'] * b2.nF
gL = nfs_param['gL (usiemens)'] * b2.usiemens
v_rest = nfs_param['v_rest (mV)'] * b2.mV
v_reset = nfs_param['v_reset (mV)'] * b2.mV
tau_refr = nfs_param['tau_refr (ms)'] * b2.ms

v_thresh = nfs_param['v_thresh (mV)'] * b2.mV
del_v = nfs_param['del_v (mV)'] * b2.mV
amp_w1 = nfs_param['amp_w1 (nA)'] * b2.nA
tau_w1 = nfs_param['tau_w1 (ms)'] * b2.ms
amp_w2 = nfs_param['amp_w2 (nA)'] * b2.nA
tau_w2 = nfs_param['tau_w2 (ms)'] * b2.ms

amp_vt1 = nfs_param['amp_vt1 (mV)'] * b2.mV
tau_vt1 = nfs_param['tau_vt1 (ms)'] * b2.ms
amp_vt2 = nfs_param['amp_vt2 (mV)'] * b2.mV
tau_vt2 = nfs_param['tau_vt2 (ms)'] * b2.ms

In [7]:
## amplitude to be played with 
# I_ext = get_step_current(t_start = 100, t_end = 700, unit_time = 1*b2.ms, amplitude = 0.08*b2.namp)
I_ext = get_ou_current(1,Delta_sigma=2.0,sigma_0=10.0)
rate_0 = b2.defaultclock.dt/b2.ms * b2.Hz
time = 1000 * b2.ms
lambda_0 = 10 * b2.kHz
VIP = b2.NeuronGroup(1, model = model_eqs, reset = reset_eqs, threshold = "v > vt",refractory = tau_refr, method = 'rk4')
VIP.v = v_rest
voltage_monitor_vip = b2.StateMonitor(VIP, ['v','vt','w1','w2'], record=True)
spike_monitor_vip = b2.SpikeMonitor(VIP)

vip_monitors = [voltage_monitor_vip, spike_monitor_vip]
print("Before simulation" , datetime.datetime.now())

vip_net = b2.Network(VIP)
vip_net.add(vip_monitors)
vip_net.run(time)

print("After simulation" , datetime.datetime.now())

In [8]:
fig2 = plt.figure(figsize=(5,10))
plot_voltage_and_current_traces(voltage_monitor_vip, spike_monitor_vip, I_ext,title = 'NFS neuron')
plt.show()

In [9]:
# fig2.savefig('../figs/25_nfsou.jpg')